**Retrieving Data (from previous notebook)**

In [52]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
website_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(website_url,'lxml')
w_table = soup.find('table',{'class':'wikitable sortable'})
list_contents = [rl.getText().strip().split('\n') for rl in w_table.findAll('tr')]
data = pd.DataFrame(list_contents)
columns = data.iloc[0]
data = data[1:]
data.columns = columns
data_filtered = data.loc[data['Borough'] != 'Not assigned']
data_grouped = data_filtered.groupby(['Postcode', 'Borough']).agg(lambda x: ','.join(set(x)) )
data_grouped.reset_index(level=[0,1], inplace=True)
data_grouped['Neighbourhood'] = data_grouped.apply(lambda x: x[2] if x[2] != 'Not assigned' else x[1], axis=1)
canada_post_table = data_grouped
canada_post_table.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern,Rouge"
1,M1C,Scarborough,"Port Union,Rouge Hill,Highland Creek"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [53]:
# !conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values
# !conda install -c conda-forge geocoder --yes 
import geocoder

In [54]:
def findCoordi(post_code):
    lat_lng = None
    postal_code = post_code
    address = '{}, Toronto, Ontario'.format(postal_code)
    g = geocoder.arcgis(address)
    lat_lng = g.latlng
    latitude = lat_lng[0]
    longitude = lat_lng[1]
    return lat_lng

**Filter and extract location coordinates**

In [56]:
toronto_data = canada_post_table.loc[canada_post_table['Borough'].str.contains('.*Toronto', regex=True)]
toronto_data['coordi'] = toronto_data['Postcode'].apply(findCoordi)
toronto_data['latitude'] = toronto_data['coordi'].str.get(0)
toronto_data['longtiude'] = toronto_data['coordi'].str.get(1)
toronto_data.reset_index(inplace=True)
toronto_data.drop('coordi', axis=1, inplace=True)
toronto_data.drop('index', axis=1, inplace=True)
toronto_data

//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docume

,Postcode,Borough,Neighbourhood,latitude,longtiude
0,M4E,East Toronto,The Beaches,43.676845,-79.295225
1,M4K,East Toronto,"The Danforth West,Riverdale",43.683262,-79.355120
2,M4L,East Toronto,"The Beaches West,India Bazaar",43.667965,-79.314673
3,M4M,East Toronto,Studio District,43.662766,-79.334830
4,M4N,Central Toronto,Lawrence Park,43.728160,-79.387085
5,M4P,Central Toronto,Davisville North,43.712815,-79.388526
6,M4R,Central Toronto,North Toronto West,43.714523,-79.406960
7,M4S,Central Toronto,Davisville,43.703395,-79.385964
8,M4T,Central Toronto,"Moore Park,Summerhill East",43.690655,-79.383561
9,M4V,Central Toronto,"Summerhill West,Rathnelly,Forest Hill SE,Deer ...",43.686083,-79.402335
